# Get Lexicon

For the `20210515` run, we just used re-used the lexicon from `20200224`. Also, we didn't try filtering out non-human pathways this time, unlike the process we used for `20200224`, where we excluded figures from papers where PubTator had identified at least one species and none of those species were human or mouse.

For the next run, it would be good to finish this notebook to get an updated lexicon and get species information for the papers. [The notes from our old codebook](https://github.com/wikipathways/pathway-figure-ocr/blob/76ae219e65bcc723bbeaa3b82744b4aec78fc9d7/codebook.md#generating-files-and-initial-tables) may be useful for how to do this.

In [4]:
%load_ext sql

In [1]:
import csv
import hashlib
import io
import json
import os
import re
import subprocess
import sys
import warnings
from itertools import zip_longest
from pathlib import Path, PurePath
from pprint import pprint

import numpy as np
import pandas as pd
import requests
import requests_cache
import seaborn as sns

# from wand.image import Image
from IPython.display import Image
from nltk.metrics import edit_distance

In [35]:
target_date = "20210513"
images_dir = Path(f"../data/images/{target_date}")

## Old Lexicon

In [30]:
%sql postgresql:///pfocr20200224

In [33]:
%%sql lexicon2020 << SELECT symbol, xref as ncbigene_id, source
FROM lexicon
INNER JOIN xrefs ON lexicon.xref_id = xrefs.id
INNER JOIN symbols ON lexicon.symbol_id = symbols.id;

 * postgresql:///pfocr20200224
60244 rows affected.
Returning data to local variable lexicon2020


In [34]:
lexicon2020_df = lexicon2020.DataFrame()
lexicon2020_df

,symbol,ncbigene_id,source
0,S863-7,2,hgnc_alias_symbol
1,FWP007,2,hgnc_alias_symbol
2,CPAMD5,2,hgnc_alias_symbol
3,A2M,2,hgnc_symbol
4,AAC2,10,hgnc_prev_symbol
...,...,...,...
60239,C17orf113,110806298,hgnc_symbol
60240,CSNKA2IP,111064647,hgnc_symbol
60241,C19orf85,111064650,hgnc_symbol
60242,IFITMD9,111216276,hgnc_alias_symbol


In [37]:
with open(images_dir.joinpath("lexicon2020.json"), "w") as f:
    lexicon2020_df.to_json(f)

## Updated Lexicon

I'm trying to translate [these steps](https://github.com/wikipathways/pathway-figure-ocr/blob/7b34d23982c7812ec98256ab1caf8c6ca3abccbb/codebook.md#generating-files-and-initial-tables) into Python in order to build a list of gene symbols to look for in the OCR text, but I haven't finished this yet. Status of the three main sources:

- HGNC: done
- bioentities: in progress
- WikiPathways: not yet started

For the bioentities file, it appears there are inconsistencies in mapping bioentities to NCBI Gene IDs. See that section below for more details.

In [2]:
print(requests_cache.__file__)

/nix/store/ig1jyhw2kyjy1hb13zfzmkqqdmgi94mp-python3-3.8.8-env/lib/python3.8/site-packages/requests_cache/__init__.py


In [3]:
requests_cache.install_cache("pfocr_cache")

In [4]:
%load_ext sql

In [5]:
sns.set_style("dark")

In [6]:
import Levenshtein
import Levenshtein_search
import polyleven

## HGNC

In [7]:
hgnc_data_url = "https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_app_sym&col=gd_app_name&col=gd_status&col=gd_prev_sym&col=gd_aliases&col=gd_pub_acc_ids&col=gd_locus_type&col=gd_date_mod&col=family.id&col=gd_locus_group&col=gd_name_aliases&col=gd_date_sym_change&col=gd_pub_eg_id&col=family.name&col=gd_date_name_change&col=gd_prev_name&col=gd_date2app_or_res&status=Approved&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit"
hgnc_data = []
r = requests.get(hgnc_data_url, stream=True)
for record in csv.DictReader(
    (line.decode("utf-8") for line in r.iter_lines()), delimiter="\t"
):
    hgnc_data.append(record)

In [8]:
print(len(hgnc_data))
hgnc_data[:2]

42549


[{'HGNC ID': 'HGNC:5',
  'Approved symbol': 'A1BG',
  'Approved name': 'alpha-1-B glycoprotein',
  'Status': 'Approved',
  'Previous symbols': '',
  'Alias symbols': '',
  'Accession numbers': '',
  'Locus type': 'gene with protein product',
  'Date modified': '2020-09-17',
  'Gene group ID': '594',
  'Locus group': 'protein-coding gene',
  'Alias names': '',
  'Date symbol changed': '',
  'NCBI Gene ID': '1',
  'Gene group name': 'Immunoglobulin like domain containing',
  'Date name changed': '',
  'Previous name': '',
  'Date approved': '1989-06-30'},
 {'HGNC ID': 'HGNC:37133',
  'Approved symbol': 'A1BG-AS1',
  'Approved name': 'A1BG antisense RNA 1',
  'Status': 'Approved',
  'Previous symbols': 'NCRNA00181, A1BGAS, A1BG-AS',
  'Alias symbols': 'FLJ23569',
  'Accession numbers': 'BC040926',
  'Locus type': 'RNA, long non-coding',
  'Date modified': '2013-06-27',
  'Gene group ID': '1987',
  'Locus group': 'non-coding RNA',
  'Alias names': '',
  'Date symbol changed': '2010-11-25',

In [9]:
raw_hgnc_df = (
    pd.DataFrame(hgnc_data)[
        [
            "HGNC ID",
            "NCBI Gene ID",
            # "Status",
            "Approved symbol",
            "Approved name",
            "Alias symbols",
            "Alias names",
            "Previous symbols",
            "Previous name",
        ]
    ]
    .rename(
        columns={
            "HGNC ID": "hgnc_id",
            "NCBI Gene ID": "ncbigene_id",
            "Approved symbol": "approved_symbol",
            "Approved name": "approved_name",
            "Alias symbols": "alias_symbols",
            "Alias names": "alias_names",
            "Previous symbols": "previous_symbols",
            "Previous name": "previous_name",
        }
    )
    .replace("", np.nan)
)
print(len(raw_hgnc_df))
# raw_hgnc_df

42549


"ambiguous symbols" are those that map to multiple ncbigene ids

- `94436` when keeping first approved ambiguous symbol
- `92535` when dropping all rows with ambiguous symbols

In [10]:
hgnc_df = raw_hgnc_df[raw_hgnc_df["ncbigene_id"].notnull()]

name_columns = ["approved_name", "previous_name", "alias_name"]

hgnc_df = (
    hgnc_df.assign(
        alias_symbol=hgnc_df["alias_symbols"].str.split(", "),
        alias_name=hgnc_df["alias_names"].str.split(", "),
        previous_symbol=hgnc_df["previous_symbols"].str.split(", "),
    )
    .explode("alias_symbol")
    .explode("alias_name")
    .explode("previous_symbol")
    .drop(
        columns=["alias_symbols", "alias_names", "previous_symbols"]
        + name_columns
    )
    .melt(
        id_vars=["hgnc_id", "ncbigene_id"],
        value_vars=["approved_symbol", "alias_symbol", "previous_symbol"],
        var_name="source",
        value_name="symbol",
    )
    .dropna()
    # drop exact duplicates
    .drop_duplicates()
)

hgnc_df = hgnc_df[
    (hgnc_df["source"] == "approved_symbol")
    | (hgnc_df["symbol"].str.len() >= 3)
]

variables_by_priority = {
    "approved_symbol": 0,
    "previous_symbol": 1,
    "alias_symbol": 2,
}

hgnc_df["priority"] = hgnc_df["source"].apply(
    lambda x: variables_by_priority[x]
)

# For rows with ambiguous symbols, keep the first for every symbol and drop the rest.
# Note the sorting is approved before previous before alias.
hgnc_df = (
    hgnc_df.sort_values(by=["priority"])
    .drop_duplicates(subset=["symbol"])
    .drop(columns=["priority"])
)

# we don't want this
# drop all rows with ambiguous symbols
# hgnc_df = hgnc_df.drop_duplicates(subset=["value"], keep=False)

hgnc_df

,hgnc_id,ncbigene_id,source,symbol
0,HGNC:5,1,approved_symbol,A1BG
58949,HGNC:42824,100873287,approved_symbol,RNA5SP48
58950,HGNC:42825,100873288,approved_symbol,RNA5SP49
58951,HGNC:42827,106478992,approved_symbol,RNA5SP50
58952,HGNC:42828,100873289,approved_symbol,RNA5SP51
...,...,...,...,...
139992,HGNC:15864,10616,alias_symbol,HOIL1
139993,HGNC:26120,79863,alias_symbol,FLJ21172
139994,HGNC:26120,79863,alias_symbol,HsT169
140024,HGNC:28842,83759,alias_symbol,MGC10871


## Bioentities

In [11]:
bioentities_data_url = "https://raw.githubusercontent.com/wikipathways/bioentities/master/relations.csv"
bioentities_data = []
r = requests.get(bioentities_data_url, stream=True)
for record in csv.DictReader(
    (line.decode("utf-8") for line in r.iter_lines()),
    fieldnames=["type", "symbol", "isa", "type2", "bioentity"],
):
    bioentities_data.append(record)

bioentities_df = pd.DataFrame(bioentities_data)
# There are 9 entries from UP (UniProt). They are duplicates the HGNC entries, so we ignore them.
bioentities_df = (
    bioentities_df[
        bioentities_df["type"].isin(
            {
                "HGNC",
                "BE",
            }
        )
    ]
).drop(columns=["isa", "type2"])
bioentities_df

,type,symbol,bioentity
0,HGNC,EIF4EBP1,EIF4EBP
1,HGNC,EIF4EBP2,EIF4EBP
2,HGNC,EIF4EBP3,EIF4EBP
3,HGNC,ACTA1,Actin
4,HGNC,ACTA2,Actin
...,...,...,...
2305,HGNC,ITGB4,Integrin
2306,HGNC,ITGB5,Integrin
2307,HGNC,ITGB6,Integrin
2308,HGNC,ITGB7,Integrin


There are no missing values:

In [12]:
print(bioentities_df.replace("", np.nan).isna().sum().sum())
bioentities_df.replace("", np.nan).isna().any()

0


type         False
symbol       False
bioentity    False
dtype: bool

In [13]:
joined_df = bioentities_df.merge(
    hgnc_df[["ncbigene_id", "symbol"]], how="left", on="symbol"
)
joined_df

,type,symbol,bioentity,ncbigene_id
0,HGNC,EIF4EBP1,EIF4EBP,1978
1,HGNC,EIF4EBP2,EIF4EBP,1979
2,HGNC,EIF4EBP3,EIF4EBP,8637
3,HGNC,ACTA1,Actin,58
4,HGNC,ACTA2,Actin,59
...,...,...,...,...
2296,HGNC,ITGB4,Integrin,3691
2297,HGNC,ITGB5,Integrin,3693
2298,HGNC,ITGB6,Integrin,3694
2299,HGNC,ITGB7,Integrin,3695


In [14]:
set(joined_df["type"].to_list())

{'BE', 'HGNC'}

In [15]:
joined_df[joined_df["ncbigene_id"].isnull()]

,type,symbol,bioentity,ncbigene_id
9,BE,Activin_A,Activin,NaN
10,BE,Activin_AB,Activin,NaN
11,BE,Activin_B,Activin,NaN
12,BE,Inhibin_A,Inhibin,NaN
13,BE,Inhibin_B,Inhibin,NaN
...,...,...,...,...
2127,BE,Troponin_I,Troponin,NaN
2128,BE,Troponin_T,Troponin,NaN
2183,HGNC,GRIN2A,NMDAR,NaN
2240,BE,MAP2K,MAPKK,NaN


In [16]:
joined_df[joined_df["bioentity"] == "AMPK_alpha"]

,type,symbol,bioentity,ncbigene_id
28,HGNC,PRKAA1,AMPK_alpha,5562
29,HGNC,PRKAA2,AMPK_alpha,5563


In [17]:
joined_df[joined_df["symbol"] == "HSP90A"]

,type,symbol,bioentity,ncbigene_id
231,BE,HSP90A,HSP90,NaN


In [18]:
def get_hierarchy_levels(bioentity, hierarchy_levels=None):
    # ******
    # The bioentities appear to be at least sometimes in a hierarchical
    # relationship. This function traces the path up the hierarchy for
    # a bioentity in joined_df.
    # ******

    if not hierarchy_levels:
        # make the trace include the lowest (first) item
        hierarchy_levels = bioentity

    # We're trying to go one level up the hierarchy.
    # If this bioentity has a parent bioentity, then this bioentity will be
    # listed as a symbol in a row where the parent bioentity is listed in the
    # bioentity column. I don't know whether we should expect multiple results.
    match_column = joined_df[joined_df["symbol"] == bioentity]["bioentity"]
    if len(match_column) > 0:
        ancestor = match_column.to_list()[0]

        if ancestor in hierarchy_levels.split(","):
            return hierarchy_levels
        # TODO: when I use a list for hierarchy_levels, the column values in the df end
        # up all being the same, but when I use a string, they each properly
        # show the different trace for their specific row. Why?
        # if ancestor in hierarchy_levels:
        #    return hierarchy_levels
        # hierarchy_levels.insert(0, ancestor)

        return get_hierarchy_levels(
            ancestor, ",".join([ancestor, hierarchy_levels])
        )

    return hierarchy_levels


joined_df["hierarchy_levels"] = joined_df["bioentity"].apply(
    # TODO: see comment in fn about str vs. list
    lambda x: get_hierarchy_levels(x).split(",")
)

joined_df = joined_df.assign(
    hierarchy_apex=joined_df["hierarchy_levels"].apply(lambda x: x[0]),
    level_number=joined_df["hierarchy_levels"].apply(lambda x: len(x)),
)

joined_df

,type,symbol,bioentity,ncbigene_id,hierarchy_levels,hierarchy_apex,level_number
0,HGNC,EIF4EBP1,EIF4EBP,1978,[EIF4EBP],EIF4EBP,1
1,HGNC,EIF4EBP2,EIF4EBP,1979,[EIF4EBP],EIF4EBP,1
2,HGNC,EIF4EBP3,EIF4EBP,8637,[EIF4EBP],EIF4EBP,1
3,HGNC,ACTA1,Actin,58,[Actin],Actin,1
4,HGNC,ACTA2,Actin,59,[Actin],Actin,1
...,...,...,...,...,...,...,...
2296,HGNC,ITGB4,Integrin,3691,[Integrin],Integrin,1
2297,HGNC,ITGB5,Integrin,3693,[Integrin],Integrin,1
2298,HGNC,ITGB6,Integrin,3694,[Integrin],Integrin,1
2299,HGNC,ITGB7,Integrin,3695,[Integrin],Integrin,1


In [19]:
joined_df["level_number"].max()

3

In [20]:
joined_df[joined_df["level_number"] == 3]

,type,symbol,bioentity,ncbigene_id,hierarchy_levels,hierarchy_apex,level_number
233,HGNC,HSP90AA1,HSP90AA,3320,"[HSP90, HSP90A, HSP90AA]",HSP90,3
623,HGNC,MAP2K1,MEK,5604,"[MAPKK, MAP2K, MEK]",MAPKK,3
624,HGNC,MAP2K2,MEK,5605,"[MAPKK, MAP2K, MEK]",MAPKK,3
1428,HGNC,PIK3R1,PI3K_p85,5295,"[PI3K, PIK3R_I, PI3K_p85]",PI3K,3
1429,HGNC,PIK3R2,PI3K_p85,5296,"[PI3K, PIK3R_I, PI3K_p85]",PI3K,3


I'm not sure how to interpret the following pattern in the bioentities, but it appears to be a mistake. As far as I can tell, `MAP2K`, `MAPKK` and `MEK` are synonyms, and every one of them each refers to the following collection of NCBI Gene IDs: `5604`, `5605`, `5606`, `5607`, `5608`, `5609`, `6416`. But `STAT` and `STAT5` are not synonyms. `STAT` refers to the NCBI Gene IDs `6772`, `6773`, `6774`, `6775`, `6776`, `6776`, `6777`, `6778`, but `STAT5` only refers to `6776` and `6777`.

In the df, we have the following `BE` relationships for rows with hierarchy apex `MAPKK`:
- `MAPKK` & `MAP2K`
- `MAP2K` & `MEK`

and for rows with hierarchy apex `STAT`:
- `STAT` & `STAT5`

If `BE` indicates synonyms, we get the correct results for `MAP2K`, `MAPKK` and `MEK` but the wrong results for `STAT` and `STAT5`.

In [21]:
joined_df[(joined_df["hierarchy_apex"] == "MAPKK")]

,type,symbol,bioentity,ncbigene_id,hierarchy_levels,hierarchy_apex,level_number
623,HGNC,MAP2K1,MEK,5604,"[MAPKK, MAP2K, MEK]",MAPKK,3
624,HGNC,MAP2K2,MEK,5605,"[MAPKK, MAP2K, MEK]",MAPKK,3
625,BE,MEK,MAP2K,NaN,"[MAPKK, MAP2K]",MAPKK,2
626,HGNC,MAP2K3,MAP2K,5606,"[MAPKK, MAP2K]",MAPKK,2
627,HGNC,MAP2K4,MAP2K,6416,"[MAPKK, MAP2K]",MAPKK,2
628,HGNC,MAP2K5,MAP2K,5607,"[MAPKK, MAP2K]",MAPKK,2
629,HGNC,MAP2K6,MAP2K,5608,"[MAPKK, MAP2K]",MAPKK,2
630,HGNC,MAP2K7,MAP2K,5609,"[MAPKK, MAP2K]",MAPKK,2
2240,BE,MAP2K,MAPKK,NaN,[MAPKK],MAPKK,1


In [22]:
joined_df[
    (joined_df["hierarchy_apex"] == "MAPKK") & (joined_df["type"] == "BE")
][["bioentity", "type", "symbol"]]

,bioentity,type,symbol
625,MAP2K,BE,MEK
2240,MAPKK,BE,MAP2K


In [23]:
joined_df[(joined_df["hierarchy_apex"] == "MAPKK")][
    "ncbigene_id"
].sort_values().dropna().to_list()

['5604', '5605', '5606', '5607', '5608', '5609', '6416']

In [24]:
joined_df[(joined_df["hierarchy_apex"] == "STAT")]

,type,symbol,bioentity,ncbigene_id,hierarchy_levels,hierarchy_apex,level_number
283,HGNC,STAT1,STAT,6772,[STAT],STAT,1
284,HGNC,STAT2,STAT,6773,[STAT],STAT,1
285,HGNC,STAT3,STAT,6774,[STAT],STAT,1
286,HGNC,STAT4,STAT,6775,[STAT],STAT,1
287,HGNC,STAT5A,STAT5,6776,"[STAT, STAT5]",STAT,2
288,HGNC,STAT5B,STAT5,6777,"[STAT, STAT5]",STAT,2
289,HGNC,STAT6,STAT,6778,[STAT],STAT,1
290,BE,STAT5,STAT,6776,[STAT],STAT,1


In [25]:
joined_df[
    (joined_df["hierarchy_apex"] == "STAT") & (joined_df["type"] == "BE")
][["bioentity", "type", "symbol"]]

,bioentity,type,symbol
290,STAT,BE,STAT5


In [26]:
joined_df[(joined_df["hierarchy_apex"] == "STAT")][
    "ncbigene_id"
].sort_values().dropna().to_list()

['6772', '6773', '6774', '6775', '6776', '6776', '6777', '6778']

## OLD

In [25]:
joined_df[(joined_df["bioentities"].str.startswith("STAT"))]

,type,symbol,bioentities,ncbigene_id,hierarchy_path,top_ancestor,hierarchy_levels,hierarchy_apex,level_number
283,HGNC,STAT1,STAT,6772,[STAT],STAT,[STAT],STAT,1
284,HGNC,STAT2,STAT,6773,[STAT],STAT,[STAT],STAT,1
285,HGNC,STAT3,STAT,6774,[STAT],STAT,[STAT],STAT,1
286,HGNC,STAT4,STAT,6775,[STAT],STAT,[STAT],STAT,1
287,HGNC,STAT5A,STAT5,6776,"[STAT, STAT5]",STAT,"[STAT, STAT5]",STAT,2
288,HGNC,STAT5B,STAT5,6777,"[STAT, STAT5]",STAT,"[STAT, STAT5]",STAT,2
289,HGNC,STAT6,STAT,6778,[STAT],STAT,[STAT],STAT,1
290,BE,STAT5,STAT,6776,[STAT],STAT,[STAT],STAT,1


In [331]:
joined_df[(joined_df["bioentities"].str.startswith("STAT"))]

,type,symbol,bioentities,ncbigene_id,genealogy,genealogy_chunks,top_ancestor
283,HGNC,STAT1,STAT,6772,STAT,[STAT],STAT
284,HGNC,STAT2,STAT,6773,STAT,[STAT],STAT
285,HGNC,STAT3,STAT,6774,STAT,[STAT],STAT
286,HGNC,STAT4,STAT,6775,STAT,[STAT],STAT
287,HGNC,STAT5A,STAT5,6776,"STAT,STAT5","[STAT, STAT5]",STAT
288,HGNC,STAT5B,STAT5,6777,"STAT,STAT5","[STAT, STAT5]",STAT
289,HGNC,STAT6,STAT,6778,STAT,[STAT],STAT
290,BE,STAT5,STAT,6776,STAT,[STAT],STAT


MAP2K == MEK == MAPKK

In [335]:
joined_df[
    (joined_df["bioentities"] == "MAP2K")
    | (joined_df["bioentities"] == "MAPKK")
    | (joined_df["bioentities"] == "MEK")
    | (joined_df["symbol"].str.startswith("MAP2K"))
    | (joined_df["symbol"].str.startswith("MAPKK"))
    | (joined_df["symbol"].str.startswith("MEK"))
]

,type,symbol,bioentities,ncbigene_id,genealogy,genealogy_chunks,top_ancestor
623,HGNC,MAP2K1,MEK,5604,"MAPKK,MAP2K,MEK","[MAPKK, MAP2K, MEK]",MAPKK
624,HGNC,MAP2K2,MEK,5605,"MAPKK,MAP2K,MEK","[MAPKK, MAP2K, MEK]",MAPKK
625,BE,MEK,MAP2K,NaN,"MAPKK,MAP2K","[MAPKK, MAP2K]",MAPKK
626,HGNC,MAP2K3,MAP2K,5606,"MAPKK,MAP2K","[MAPKK, MAP2K]",MAPKK
627,HGNC,MAP2K4,MAP2K,6416,"MAPKK,MAP2K","[MAPKK, MAP2K]",MAPKK
628,HGNC,MAP2K5,MAP2K,5607,"MAPKK,MAP2K","[MAPKK, MAP2K]",MAPKK
629,HGNC,MAP2K6,MAP2K,5608,"MAPKK,MAP2K","[MAPKK, MAP2K]",MAPKK
630,HGNC,MAP2K7,MAP2K,5609,"MAPKK,MAP2K","[MAPKK, MAP2K]",MAPKK
2240,BE,MAP2K,MAPKK,NaN,MAPKK,[MAPKK],MAPKK


In [330]:
joined_df[joined_df["bioentities"] == joined_df["top_ancestor"]]

,type,symbol,bioentities,ncbigene_id,genealogy,genealogy_chunks,top_ancestor
0,HGNC,EIF4EBP1,EIF4EBP,1978,EIF4EBP,[EIF4EBP],EIF4EBP
1,HGNC,EIF4EBP2,EIF4EBP,1979,EIF4EBP,[EIF4EBP],EIF4EBP
2,HGNC,EIF4EBP3,EIF4EBP,8637,EIF4EBP,[EIF4EBP],EIF4EBP
3,HGNC,ACTA1,Actin,58,Actin,[Actin],Actin
4,HGNC,ACTA2,Actin,59,Actin,[Actin],Actin
...,...,...,...,...,...,...,...
2296,HGNC,ITGB4,Integrin,3691,Integrin,[Integrin],Integrin
2297,HGNC,ITGB5,Integrin,3693,Integrin,[Integrin],Integrin
2298,HGNC,ITGB6,Integrin,3694,Integrin,[Integrin],Integrin
2299,HGNC,ITGB7,Integrin,3695,Integrin,[Integrin],Integrin


In [312]:
joined_df[
    (joined_df["genealogy"].str.contains(","))
    & (joined_df["genealogy"].notnull())
]

,type,symbol,bioentities,ncbigene_id,genealogy
14,HGNC,INHBA,Activin_A,3624,"Activin,Activin_A"
15,HGNC,INHBA,Activin_AB,3624,"Activin,Activin_AB"
16,HGNC,INHBB,Activin_AB,3625,"Activin,Activin_AB"
17,HGNC,INHBB,Activin_B,3625,"Activin,Activin_B"
18,HGNC,INHA,Inhibin_A,3623,"Inhibin,Inhibin_A"
...,...,...,...,...,...
2158,HGNC,CDKN1C,CDKN1,1028,"CDKN,CDKN1"
2159,HGNC,CDKN2A,CDKN2,1029,"CDKN,CDKN2"
2160,HGNC,CDKN2B,CDKN2,1030,"CDKN,CDKN2"
2161,HGNC,CDKN2C,CDKN2,1031,"CDKN,CDKN2"


In [265]:
i = 0
myd = dict()

for bioentities, df in joined_df[joined_df["type"] == "BE"].groupby(
    "bioentities"
):
    result = get_parents(bioentities)
    if len(result) > 0:
        print(result)
    #    print(bioentities)
    #    print(df)
    #    mysd = dict()
    #    myd["bioentities"] = mysd

    #    parent = joined_df[joined_df["symbol"] == bioentities]
    #    has_parent = len(parent) > 0
    #    print(has_parent)

    #    for s in df["symbol"].to_list():
    #        children = joined_df[joined_df["bioentities"] == s]
    #        mysd["children"] = children
    #        has_children = len(children) > 0
    #        print(has_children)
    #        # print(j)
    #        # print(s)
    #        # display(joined_df[joined_df["bioentities"] == s])
    i += 1
    if int(i) > 100:
        break

AMPK
AP1
ATP_synthase
Actin
Activin
Apolipoprotein
CAMK
CDKN
COX
Caspase
Chemokine
Cyclin
GRK
Galpha
HDAC
HIF
HSP90
HSP90A
['HSP90']
HSP90
['HSP90']
Inhibin
['HSP90']
Interferon
['HSP90']
MAP2K
['MAPKK']
MAPKK
['MAPKK', 'HSP90']
MAP3K
['MAPKK', 'HSP90']
MAPK
['MAPKK', 'HSP90']
MAPKK
['MAPKK', 'HSP90']
MYL
['MAPKK', 'HSP90']
NFKB
['MAPKK', 'HSP90']
NRG
['MAPKK', 'HSP90']
Na_K_ATPase
['MAPKK', 'HSP90']
PDE
['MAPKK', 'HSP90']
PI3K
['MAPKK', 'HSP90']
PIK3R_I
['PI3K']
PI3K
['PI3K', 'MAPKK', 'HSP90']
PKA
['PI3K', 'MAPKK', 'HSP90']
PLA2
['PI3K', 'MAPKK', 'HSP90']
PLC
['PI3K', 'MAPKK', 'HSP90']
PPP1
['PI3K', 'MAPKK', 'HSP90']
PPP2
['PI3K', 'MAPKK', 'HSP90']
PPP3
['PI3K', 'MAPKK', 'HSP90']
RSK
['PI3K', 'MAPKK', 'HSP90']
SMAD
['PI3K', 'MAPKK', 'HSP90']
STAT
['PI3K', 'MAPKK', 'HSP90']
Troponin
['PI3K', 'MAPKK', 'HSP90']
Tubulin
['PI3K', 'MAPKK', 'HSP90']


In [226]:
multicol1 = pd.MultiIndex.from_tuples([("weight", "kg"), ("weight", "pounds")])
df_multi_level_cols1 = pd.DataFrame(
    [[1, 2], [2, 4]], index=["cat", "dog"], columns=multicol1
)
df_multi_level_cols1

weight       
        kg pounds
cat      1      2
dog      2      4

In [227]:
df_multi_level_cols1.stack()

weight
cat kg           1
    pounds       2
dog kg           2
    pounds       4

In [229]:
multi_df = joined_df.copy().set_index(["bioentities", "symbol"])
multi_df

type ncbigene_id
bioentities symbol                    
EIF4EBP     EIF4EBP1  HGNC        1978
            EIF4EBP2  HGNC        1979
            EIF4EBP3  HGNC        8637
Actin       ACTA1     HGNC          58
            ACTA2     HGNC          59
...                    ...         ...
Integrin    ITGB4     HGNC        3691
            ITGB5     HGNC        3693
            ITGB6     HGNC        3694
            ITGB7     HGNC        3695
            ITGB8     HGNC        3696

[2301 rows x 2 columns]

In [237]:
multi_df.loc["STAT"]

,type,ncbigene_id
symbol,,
STAT1,HGNC,6772
STAT2,HGNC,6773
STAT3,HGNC,6774
STAT4,HGNC,6775
STAT6,HGNC,6778
STAT5,BE,6776


In [230]:
multi_df.columns

Index(['type', 'ncbigene_id'], dtype='object')

In [234]:
multi_df.index[0][0]

'EIF4EBP'

In [225]:
joined_df[joined_df["type"] == "BE"]

,type,symbol,bioentities,ncbigene_id
9,BE,Activin_A,Activin,NaN
10,BE,Activin_AB,Activin,NaN
11,BE,Activin_B,Activin,NaN
12,BE,Inhibin_A,Inhibin,NaN
13,BE,Inhibin_B,Inhibin,NaN
...,...,...,...,...
2127,BE,Troponin_I,Troponin,NaN
2128,BE,Troponin_T,Troponin,NaN
2163,BE,CDKN1,CDKN,1026
2164,BE,CDKN2,CDKN,1029


In [222]:
joined_df[(joined_df["ncbigene_id"].notnull()) & (joined_df["type"] == "BE")]

,type,symbol,bioentities,ncbigene_id
190,BE,ERK,MAPK,2048
290,BE,STAT5,STAT,6776
374,BE,TUBB,Tubulin,203068
375,BE,TUBG,Tubulin,7283
498,BE,IFNA,Interferon,3440
499,BE,IFNB,Interferon,3456
710,BE,p38,MAPK,7965
952,BE,COX4,COX,1327
953,BE,COX6A,COX,1337
954,BE,COX6B,COX,1340


In [315]:
joined_df[joined_df["genealogy"].str.contains("Inhibin")]

,type,symbol,bioentities,ncbigene_id,genealogy
12,BE,Inhibin_A,Inhibin,NaN,Inhibin
13,BE,Inhibin_B,Inhibin,NaN,Inhibin
18,HGNC,INHA,Inhibin_A,3623,"Inhibin,Inhibin_A"
19,HGNC,INHBA,Inhibin_A,3624,"Inhibin,Inhibin_A"
20,HGNC,INHA,Inhibin_B,3623,"Inhibin,Inhibin_B"
21,HGNC,INHBB,Inhibin_B,3625,"Inhibin,Inhibin_B"


In [316]:
joined_df[joined_df["genealogy"] == joined_df["bioentities"]]

,type,symbol,bioentities,ncbigene_id,genealogy
0,HGNC,EIF4EBP1,EIF4EBP,1978,EIF4EBP
1,HGNC,EIF4EBP2,EIF4EBP,1979,EIF4EBP
2,HGNC,EIF4EBP3,EIF4EBP,8637,EIF4EBP
3,HGNC,ACTA1,Actin,58,Actin
4,HGNC,ACTA2,Actin,59,Actin
...,...,...,...,...,...
2296,HGNC,ITGB4,Integrin,3691,Integrin
2297,HGNC,ITGB5,Integrin,3693,Integrin
2298,HGNC,ITGB6,Integrin,3694,Integrin
2299,HGNC,ITGB7,Integrin,3695,Integrin


In [323]:
joined_df[joined_df["genealogy"] == joined_df["bioentities"]]

,type,symbol,bioentities,ncbigene_id,genealogy
0,HGNC,EIF4EBP1,EIF4EBP,1978,EIF4EBP
1,HGNC,EIF4EBP2,EIF4EBP,1979,EIF4EBP
2,HGNC,EIF4EBP3,EIF4EBP,8637,EIF4EBP
3,HGNC,ACTA1,Actin,58,Actin
4,HGNC,ACTA2,Actin,59,Actin
...,...,...,...,...,...
2296,HGNC,ITGB4,Integrin,3691,Integrin
2297,HGNC,ITGB5,Integrin,3693,Integrin
2298,HGNC,ITGB6,Integrin,3694,Integrin
2299,HGNC,ITGB7,Integrin,3695,Integrin


0        EIF4EBP
1        EIF4EBP
2        EIF4EBP
3          Actin
4          Actin
          ...   
2296    Integrin
2297    Integrin
2298    Integrin
2299    Integrin
2300    Integrin
Name: genealogy_chunks, Length: 2301, dtype: object

In [314]:
joined_df[
    (joined_df["bioentities"] == "Inhibin")
    | (joined_df["bioentities"] == "inhibin")
    | (joined_df["symbol"].str.startswith("Inhibin"))
    | (joined_df["symbol"].str.startswith("INHBE"))
    | (joined_df["symbol"].str.startswith("inhibin"))
]

,type,symbol,bioentities,ncbigene_id,genealogy
12,BE,Inhibin_A,Inhibin,NaN,Inhibin
13,BE,Inhibin_B,Inhibin,NaN,Inhibin


In [215]:
joined_df[joined_df["symbol"].str.startswith("Activin")]

,type,symbol,bioentities,ncbigene_id
9,BE,Activin_A,Activin,NaN
10,BE,Activin_AB,Activin,NaN
11,BE,Activin_B,Activin,NaN


0

In [173]:
set(bioentities_df["type"].to_list())

{'BE', 'HGNC', 'UP'}

In [194]:
bioentities_df[bioentities_df["type"] == "BE"]

,type,symbol,bioentities
9,BE,Activin_A,Activin
10,BE,Activin_AB,Activin
11,BE,Activin_B,Activin
12,BE,Inhibin_A,Inhibin
13,BE,Inhibin_B,Inhibin
...,...,...,...
2136,BE,Troponin_I,Troponin
2137,BE,Troponin_T,Troponin
2172,BE,CDKN1,CDKN
2173,BE,CDKN2,CDKN


In [197]:
bioentities_df[bioentities_df["bioentities"] == "CDKN"]

,type,symbol,bioentities
2172,BE,CDKN1,CDKN
2173,BE,CDKN2,CDKN
2174,HGNC,CDKN3,CDKN


In [199]:
hgnc_df[hgnc_df["value"] == "CDKN1"]

,hgnc,ncbigene,variable,value
176696,HGNC:1784,1026,previous_symbol,CDKN1


In [204]:
ncbigene_ids_by_symbol = (
    hgnc_df[["ncbigene_id", "value"]]
    .set_index("value")
    .to_dict()["ncbigene_id"]
)

{'A1BG': '1',
 'RNA5SP48': '100873287',
 'RNA5SP49': '100873288',
 'RNA5SP50': '106478992',
 'RNA5SP51': '100873289',
 'RNA5SP52': '100873290',
 'RNA5SP53': '100873291',
 'RNA5SP54': '100873292',
 'RNA5SP55': '100873293',
 'RNA5SP56': '100873294',
 'RNA5SP57': '106480754',
 'RNA5SP59': '106480357',
 'RNA5SP60': '100873295',
 'RNA5SP61': '100873296',
 'RNA5SP62': '100873297',
 'RNA5SP63': '100873298',
 'RNA5SP47': '100873286',
 'RNA5SP46': '100873285',
 'RNA5SP45': '100873284',
 'RNA5SP44': '100873283',
 'RNA5SP27': '100873359',
 'RNA5SP28': '100873360',
 'RNA5SP29': '100873361',
 'RNA5SP30': '100873362',
 'RNA5SP31': '100873363',
 'RNA5SP33': '100873365',
 'RNA5SP34': '100873366',
 'RNA5SP64': '100873299',
 'RNA5SP35': '100873367',
 'RNA5SP37': '100873369',
 'RNA5SP38': '100873370',
 'RNA5SP39': '100873371',
 'RNA5SP40': '100873279',
 'RNA5SP41': '100873280',
 'RNA5SP42': '100873281',
 'RNA5SP43': '100873282',
 'RNA5SP36': '100873368',
 'RNA5SP26': '100873358',
 'RNA5SP65': '100873300'

In [200]:
hgnc_df[hgnc_df["hgnc"] == "HGNC:1784"]

,hgnc,ncbigene,variable,value
9942,HGNC:1784,1026,approved_symbol,CDKN1A
176696,HGNC:1784,1026,previous_symbol,CDKN1
93319,HGNC:1784,1026,alias_symbol,P21
93325,HGNC:1784,1026,alias_symbol,p21Cip1/Waf1
93324,HGNC:1784,1026,alias_symbol,p21CIP1
93323,HGNC:1784,1026,alias_symbol,CAP20
93322,HGNC:1784,1026,alias_symbol,SDI1
93321,HGNC:1784,1026,alias_symbol,WAF1
93320,HGNC:1784,1026,alias_symbol,CIP1


In [198]:
bioentities_df[bioentities_df["symbol"] == "CDKN2"]

,type,symbol,bioentities
2173,BE,CDKN2,CDKN


In [195]:
bioentities_df[bioentities_df["bioentities"] == "Activin"]

,type,symbol,bioentities
9,BE,Activin_A,Activin
10,BE,Activin_AB,Activin
11,BE,Activin_B,Activin


In [196]:
bioentities_df[bioentities_df["symbol"] == "Activin_A"]

,type,symbol,bioentities
9,BE,Activin_A,Activin


In [174]:
bioentities_df[bioentities_df["type"] == "UP"]

,type,symbol,isa,type2,bioentities
247,UP,P0C0S8,isa,BE,Histone_H2A
249,UP,P0C5Y9,isa,BE,Histone_H2A
264,UP,P04908,isa,BE,Histone_H2A
265,UP,Q6FI13,isa,BE,Histone_H2A
266,UP,P62807,isa,BE,Histone_H2B
285,UP,P68431,isa,BE,Histone_H3
286,UP,Q71DI3,isa,BE,Histone_H3
288,UP,P84243,isa,BE,Histone_H3
290,UP,P62805,isa,BE,Histone_H4


In [175]:
bioentities_df[bioentities_df["type2"] == "UP"]

,type,symbol,isa,type2,bioentities


In [176]:
set(bioentities_df["type2"].to_list())

{'BE'}